1. **Preparing Process**

---



In [1]:
import random
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from resnet import ResNet
from time import time
import os

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True

In [2]:
def print_and_log(message, file):
    print(message)
    file.write(message + '\n')

class ApplyTransform(Dataset):
    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, idx):
        x, y = self.dataset[idx]
        return self.transform(x), y

    def __len__(self):
        return len(self.dataset)

# normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
normalize = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

TRANSFORM_TRAIN = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    normalize,
])
TRANSFORM_TEST = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

TRAIN_SET = datasets.CIFAR10(root='./data', train=True, download=True, transform=TRANSFORM_TRAIN)
TEST_SET = datasets.CIFAR10(root='./data', train=False, download=True, transform=TRANSFORM_TEST)






Files already downloaded and verified
Files already downloaded and verified


In [3]:
def test(model: nn.Module, loader: DataLoader):
    criterion = nn.CrossEntropyLoss()
    test_loss = 0
    test_correct = 0
    test_total = 0
    start_time = time()
    start_memory = torch.cuda.memory_allocated(DEVICE) / 1024 if DEVICE.type == 'cuda' else 0

    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            test_correct += (outputs.argmax(1) == targets).sum().item()
            test_total += targets.size(0)

    latency = time() - start_time
    memory_usage = torch.cuda.memory_allocated(DEVICE) / 1024 if DEVICE.type == 'cuda' else 0  # kB
    test_memory_usage = memory_usage - start_memory

    test_loss /= len(loader)
    test_acc = test_correct / test_total
    return test_loss, test_acc, latency, test_memory_usage

In [4]:
def train(w_nbits, a_nbits, lambda_val, lr=0.1, weight_decay=1e-3,
          optimizer_type='sgd', epochs=200, batch_size=128):

    # import resnet20
    # model = resnet20.ResNetCIFAR(w_nbits=w_nbits, a_nbits=a_nbits).to(DEVICE)

    model = ResNet(w_nbits=w_nbits, a_nbits=a_nbits, lambda_val=lambda_val).to(DEVICE)

    ckpt_base_filename = (
        'resnet20_cifar'
        + ('' if w_nbits is None else f'_wq{w_nbits}')
        + ('' if a_nbits is None else f'_aq{a_nbits}')
        + ('' if lambda_val is None else f'_lambda{lambda_val}')
        + '.pt'
    )
    log_filename = ckpt_base_filename.replace('.pt', '.txt')
    last_saved_filename = None

    train_loader = DataLoader(TRAIN_SET, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TEST_SET, batch_size=batch_size, shuffle=False)

    criterion = nn.CrossEntropyLoss()

    if optimizer_type.lower() == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_type.lower() == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_type.lower() == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_type.lower() == 'adagrad':
        optimizer = optim.Adagrad(model.parameters(), lr=lr, weight_decay=weight_decay)
    else:
        raise ValueError(f"Unsupported optimizer type: {optimizer_type}")

    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[50, 100, 150], gamma=0.1)

    best_test_acc = 0.0
    save_threshold_epoch = min(50, epochs // 3)


    with open(log_filename, 'w') as log_file:
        print_and_log(f'\nQuantization: weight={w_nbits} activation={a_nbits} threshold={lambda_val}, Using device: {DEVICE}', log_file)

        start_memory = torch.cuda.memory_allocated(DEVICE) / 1024 if DEVICE.type == 'cuda' else 0

        for epoch in range(epochs):
            print_and_log(f'\nEpoch {epoch+1}', log_file)
            model.train()

            train_loss = 0
            train_correct = 0
            train_total = 0

            start_time = time()

            for inputs, targets in tqdm(train_loader, desc='Train', unit='batch', ascii=True, dynamic_ncols=True):
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                train_correct += (outputs.argmax(1) == targets).sum().item()
                train_total += targets.size(0)

            train_loss /= len(train_loader)
            train_acc = train_correct / train_total

            train_latency = time() - start_time
            end_memory = torch.cuda.memory_allocated(DEVICE) / 1024 if DEVICE.type == 'cuda' else 0
            train_memory_usage = end_memory - start_memory

            print_and_log(f'LR: {optimizer.param_groups[0]["lr"]:.4e}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train Latency: {train_latency:.2f}s, Memory Usage: {train_memory_usage}KB', log_file)

            model.eval()
            test_loss, test_acc, test_latency, test_memory_usage = test(model, test_loader)
            print_and_log(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test Latency: {test_latency:.2f}s, Test Memory Usage: {test_memory_usage}KB', log_file)

            scheduler.step()

            if epoch >= save_threshold_epoch:
                if test_acc > best_test_acc:
                    best_test_acc = test_acc
                    print('Saving best model ...')
                    if last_saved_filename is not None and last_saved_filename != ckpt_base_filename:
                        os.remove(last_saved_filename)
                    ckpt_filename = f'epoch_{epoch+1}_' + ckpt_base_filename
                    torch.save(model.state_dict(), ckpt_filename)
                    last_saved_filename = ckpt_filename

    print('Saving final epoch model ...')
    torch.save(model.state_dict(), ckpt_base_filename)

In [5]:
for w_nbits in [3, 2, 1]:
    for lambda_val in [0.001, 0.004, 0.008, 0.01]:
        train(w_nbits=w_nbits, a_nbits=None, lambda_val=lambda_val)


Quantization: weight=3 activation=None threshold=0.001, Using device: cpu

Epoch 1


Train:  28%|##8       | 110/391 [00:42<01:48,  2.60batch/s]


KeyboardInterrupt: 

2. **Base Model**

---



In [2]:
train(w_nbits=None, a_nbits=None, lambda_val=None)

NameError: name 'train' is not defined

**Weight Quantization**

In [11]:
for w_nbits in [3, 2, 1]:
    train(w_nbits=w_nbits, a_nbits=None)


Quantization: weight=3 activation=None, Using device: cuda

Epoch 1


Train: 100%|##########| 391/391 [00:42<00:00,  9.28batch/s]


LR: 1.0000e-01, Train Loss: 1.5942, Train Acc: 0.4119, Train Latency: 42.13s, Memory Usage: 3.04MB
Test Loss: 1.5019, Test Acc: 0.4745, Test Latency: 5.51s, Test Memory Usage: 0.19MB

Epoch 2


Train: 100%|##########| 391/391 [00:41<00:00,  9.47batch/s]


LR: 1.0000e-01, Train Loss: 1.1398, Train Acc: 0.5906, Train Latency: 41.29s, Memory Usage: 3.04MB
Test Loss: 1.9869, Test Acc: 0.4253, Test Latency: 5.50s, Test Memory Usage: 0.19MB

Epoch 3


...

LR: 1.0000e-04, Train Loss: 0.1961, Train Acc: 0.9340, Train Latency: 31.62s, Memory Usage: 3.04MB
Test Loss: 0.3704, Test Acc: 0.8742, Test Latency: 3.67s, Test Memory Usage: 0.19MB

Epoch 199


Train: 100%|##########| 391/391 [00:31<00:00, 12.40batch/s]


LR: 1.0000e-04, Train Loss: 0.1968, Train Acc: 0.9334, Train Latency: 31.54s, Memory Usage: 3.04MB
Test Loss: 0.3243, Test Acc: 0.8922, Test Latency: 3.61s, Test Memory Usage: 0.19MB

Epoch 200


Train: 100%|##########| 391/391 [00:31<00:00, 12.43batch/s]


LR: 1.0000e-04, Train Loss: 0.1984, Train Acc: 0.9333, Train Latency: 31.45s, Memory Usage: 3.04MB
Test Loss: 0.3409, Test Acc: 0.8863, Test Latency: 3.63s, Test Memory Usage: 0.19MB
Saving final epoch model ...


**Weight & Activation**

In [15]:
for w_nbits in [1, 2, 3]:
  for a_nbits in [2, 3]:
    train(w_nbits=w_nbits, a_nbits=a_nbits)


Quantization: weight=1 activation=2, Using device: cuda

Epoch 1


Train: 100%|##########| 391/391 [00:43<00:00,  9.00batch/s]


LR: 1.0000e-01, Train Loss: 2.0697, Train Acc: 0.2566, Train Latency: 43.43s, Memory Usage: 3119.0KB
Test Loss: 1.6808, Test Acc: 0.3619, Test Latency: 5.36s, Test Memory Usage: 194.0KB

Epoch 2


Train: 100%|##########| 391/391 [00:43<00:00,  8.96batch/s]


LR: 1.0000e-01, Train Loss: 1.6915, Train Acc: 0.3608, Train Latency: 43.66s, Memory Usage: 3119.0KB
Test Loss: 1.6173, Test Acc: 0.4062, Test Latency: 5.28s, Test Memory Usage: 194.0KB

Epoch 3


...

LR: 1.0000e-03, Train Loss: 0.3898, Train Acc: 0.8664, Train Latency: 53.32s, Memory Usage: 3119.0KB
Test Loss: 0.4501, Test Acc: 0.8458, Test Latency: 7.28s, Test Memory Usage: 194.0KB

Epoch 114


Train: 100%|##########| 391/391 [00:53<00:00,  7.36batch/s]


LR: 1.0000e-03, Train Loss: 0.3866, Train Acc: 0.8666, Train Latency: 53.15s, Memory Usage: 3119.0KB
Test Loss: 0.4470, Test Acc: 0.8476, Test Latency: 7.22s, Test Memory Usage: 194.0KB

Epoch 115


Train:  82%|########2 | 321/391 [00:43<00:09,  7.35batch/s]


KeyboardInterrupt: 